In [1]:
# This notebook imports COVID-19 data from usfacts.org and merges it with John Hopkins data
# to create a dataset with following columns.

# Target Features:
# Date
# Region (County or AFB): This is the county, burough, parish, etc.
# Province (State): This is the state but we use province in case we add in international data
# Country
# Total_Confirmed :Total cumulative confirmed cases
# Total_Deaths: The total cumulative reported deaths
# Total_Recoveries: The total cumulative people who were confirmed and later recovered
# Longitude (TODO: Need to add back in)
# Latitude (TODO: Need to add back in)
# Population
# Daily_Confirmed: Total confirmed cases by day
# Daily_Deaths: Total reported deaths by day
# Proportional_Confirmed: Total confirmed bases by day per 100,000 people
# Proportional_Deaths: Total reported deaths by day per 100,000 people
# Days_Confirmed: Number of days since the daily confirmed cases exceeded a given limit (limits defined below)
# Days_Deaths: Number of days since the daily reported deaths exceeded a given limit (limits defined below)

import pandas as pd
import numpy as np
import math
from datetime import datetime, date, timedelta
import matplotlib.pyplot as plt
pd.plotting.register_matplotlib_converters()
import requests
import urllib
from bs4 import BeautifulSoup
import csv
import os.path
print ('Last run ' + datetime.now().strftime('%Y-%m-%d %H:%M'))

# TODO: Add FIPS, Latitude and Longitude columns back into the final dataset

# REVISION HISTORY
revision = 'R05C03'
# R02C00: Added the corrected population data by county (data source was US Census website)

# R03C00: Added revision number to the export filenames so we can track the source notebook
#        Added proportional metrics (e.g. confirmed cases per 100,000 people)
# R04C00: Added data for 01/21 through 03/21 from usafacts.org
#        Removed the Total_Recovered, Total_Active Columns, Longitude and Latitude columns because they are not in the usafacts.org dataset. Will add back in later.
# R04C01: Added Total_Recoveries data from Wikipedia for counties in New York. Data for other states to be added later.
# R05C00: Added Total_Recoveries data from Wikipedia for counties in New Jersey, Illinois, Maine, Maryland and South Dakota. No other states on Wikipedia have recoveries data.

Last run 2020-07-02 07:31


In [13]:
# STEP 1A: Download the data from https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/
# Download the latest file containing the confirmed cases 
url = 'https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv'
fn = 'DATA/USAFACTS/CONFIRMED/' + date.today().strftime('%Y-%m-%d') + '-USAFACTS-CONFIRMED.csv'
urllib.request.urlretrieve(url, fn)

# Read the data from the file we downloaded
cdf = pd.read_csv(fn)

# Remove spurious data records from the list
cdf = cdf.drop(cdf.columns[len(cdf.columns)-1], axis=1)
remove_list = ['Statewide Unallocated','Grand Princess Cruise Ship']
cdf = cdf[~cdf['County Name'].isin(remove_list)]

# Merge the population data into the dataframe
states = pd.read_csv('DATA/REFERENCE/us-state-abbreviations.csv')
states = states.rename(columns={'Abbreviation':'State','Name':'Province'})
cdf = pd.merge(cdf, states[['State', 'Province']], on='State')

# Clean up the county names
cdf['County Name'] = cdf['County Name'].replace(regex=[' County and City'], value='')
cdf['County Name'] = cdf['County Name'].replace(regex=[' County'], value='')
cdf['County Name'] = cdf['County Name'].replace(regex=[' City and Borough'], value='')
cdf['County Name'] = cdf['County Name'].replace(regex=[' Borough'], value='')
cdf['County Name'] = cdf['County Name'].replace(regex=[' Census Area'], value='')
cdf['County Name'] = cdf['County Name'].replace(regex=[' Parish'], value='')
cdf['County Name'] = cdf['County Name'].replace(regex=[' city'], value='')
cdf['County Name'] = cdf['County Name'].replace(regex=[' City'], value='')

# Reset the index
cdf = cdf.reset_index(drop=True)

# Write the data to file just for testing
cdf.to_csv('DATA/TEST/' + date.today().strftime('%Y-%m-%d') + '-USAFACTS-CONFIRMED-MUNGED-v3.csv',index=False)

# Download the latest file containing the number of deaths
url = 'https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_deaths_usafacts.csv'
fn = 'DATA/USAFACTS/DEATHS/' + date.today().strftime('%Y-%m-%d') + '-USAFACTS-DEATHS.csv'
urllib.request.urlretrieve(url, fn)

# Read the data from the file we downloaded
ddf = pd.read_csv(fn)

# Remove 
ddf = ddf[~ddf['County Name'].isin(remove_list)]

# Merge the population data into the dataframe
ddf = pd.merge(ddf, states[['State', 'Province']], on='State')

# Clean up the county names
ddf['County Name'] = ddf['County Name'].replace(regex=[' County and City'], value='')
ddf['County Name'] = ddf['County Name'].replace(regex=[' County'], value='')
ddf['County Name'] = ddf['County Name'].replace(regex=[' City and Borough'], value='')
ddf['County Name'] = ddf['County Name'].replace(regex=[' Borough'], value='')
ddf['County Name'] = ddf['County Name'].replace(regex=[' Census Area'], value='')
ddf['County Name'] = ddf['County Name'].replace(regex=[' Parish'], value='')
ddf['County Name'] = ddf['County Name'].replace(regex=[' city'], value='')
ddf['County Name'] = ddf['County Name'].replace(regex=[' City'], value='')

# Reset the index
ddf = ddf.reset_index(drop=True)

# Create an empty dictionary with the standard columns
df1 = pd.DataFrame(columns=['Date','Region','Province','Country','Total_Confirmed','Total_Deaths'])
dict1 = {}

# Set the default country since all the data is for the US
Country = 'US'

i = 0
# loop through the rows in the columnar dataset
for row in range(0,len(cdf)):
    #Loop through the columns and add the confirmed cases to the new file
    for col in range(4,len(cdf.columns)-1):
        dict1[i] = {'Date':cdf.columns[col],'Region':cdf.iat[row,1],'Province':cdf.iat[row,cdf.columns.get_loc("Province")],'Country':Country,'Total_Confirmed':cdf.iat[row,col],'Total_Deaths':ddf.iat[row,col]}
        i += 1

# Read the data into the datafram
df1 = pd.DataFrame.from_dict(dict1,orient='index')

# Convert Total_Deaths column to int because for some reason it comes in as float
df1.Total_Deaths = df1.Total_Deaths.astype(int)

# Write the data to file for easy use elsewhere
df1.to_csv('DATA/USAFACTS/' + date.today().strftime('%Y-%m-%d') + '-USAFACTS-MERGED-' + revision + '.csv',index=False)

# STEP 1B: Download all the JHU daily files to local disk just so we have the records

# We need create a list of string dates from 01/22 through today
start_date = date(2020,1,22)
end_date = date.today()
period = end_date - start_date
datelist = []

for dindex in range(period.days):
    currentdate = start_date + timedelta(days=dindex)
    datelist.append(currentdate.strftime('%m-%d-%Y'))

# Note: This code will overwrite any files that were downloaded earlier
for d in datelist:
    url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/' + d + '.csv'
    fn = 'DATA/JHU/DAILY/' + d + '-JHU-DAILY.csv'
    if not (os.path.exists(fn)):
        urllib.request.urlretrieve(url, fn)
        
# STEP 2: Load data from 03/22 through today
#Note: The JHU data before 03/22 does not have information by county

# Setup the date range for the first period
start_date = date(2020,3,22)
end_date = date.today()
period = end_date - start_date

# Create an empty list to hold the range of date strings
drng2 = []

# Loop through the dae range and populate the list of date strings
for dindex in range(period.days):
    currentdate = start_date + timedelta(days=dindex)
    drng2.append(currentdate.strftime('%m-%d-%Y'))

# Create an empty dataframe with the standard columns
df2 = pd.DataFrame(columns=['Date','Region','Province','Country','Total_Confirmed','Total_Deaths'])
    
# Loop through the date range and put all the date into one file
for d in drng2:
    # Load the data file for this particular date in the loop
    fn = 'DATA/JHU/DAILY/' + d + '-JHU-DAILY.csv'
    rdf = pd.read_csv(fn)
    
    # Rename and remove columns
    rdf = rdf.rename(columns={"Admin2":"Region", "Province_State":"Province", "Country_Region":"Country", "Confirmed":"Total_Confirmed", "Deaths":"Total_Deaths"})
    # We are dropping Latitude and Longitude because it isn't in the USAFACTS data files, but we might want to merge back in later
    rdf = rdf.drop(['FIPS','Last_Update','Combined_Key','Recovered','Active','Lat','Long_'],axis=1)
    
    
    # Set the date corresponding to the date of the file (i.e. filename)
    rdf["Date"] = d
    
    # Append the data to the master data file for this range
    df2 = df2.append(rdf)
    
# Convert to datetime so it sorts by date, not by string
df2.Date = pd.to_datetime(df2.Date)

# Filter out all countries except the US
df2 = df2.sort_values(by=['Date'])
df2 = df2.reset_index(drop=True)
df2 = df2[df2.Country == 'US']

# Drop all records that contain data for individual cities in the Province field (they contain a comma)
df2 = df2.sort_values(by=['Date'])
df2 = df2.reset_index(drop=True)
df2 = df2.drop(df2[df2.Province.str.contains(',')].index)

# Remove the extraneous province data
df2 = df2.sort_values(by=['Date'])
df2 = df2.reset_index(drop=True)
remove_list = ['Chicago','Unassigned Location (From Diamond Princess)','Grand Princess Cruise Ship','Grand Princess','Diamond Princess','Wuhan Evacuee','Recovered','Northern Mariana Islands','American Samoa','Guam','United States Virgin Islands','Virgin Islands','US','Puerto Rico']
df2 = df2[~df2.Province.isin(remove_list)]
df2 = df2.sort_values(by=['Date'])    
df2 = df2.reset_index(drop=True)

# Merge in the USAFACTS dataset for data prior to 03/22
df1.Date = pd.to_datetime(df1.Date)
df1b = df1[df1.Date < pd.to_datetime(start_date)]
df2 = df2.append(df1b)
df2 = df2.sort_values(by=['Date'])    
df2 = df2.reset_index(drop=True)

# Write the data to disk
dstr = date.today().strftime('%Y-%m-%d')
df2.to_csv('DATA/' + dstr + '-COVID-19-ALL-' + revision + '.csv',index=False)

# STEP 3: Load the US Census data by population and clean it up so we can merge the population data into data set
popdf = pd.read_csv('DATA/REFERENCE/us-population-by-county.csv')
popdf = popdf.drop(popdf[popdf.Province == popdf.Region].index)

#Remove the word "county" from the end of the Regions
popdf['Region'] = popdf['Region'].replace(regex=[' County'], value='')
popdf['Region'] = popdf['Region'].replace(regex=[' City and Borough'], value='')
popdf['Region'] = popdf['Region'].replace(regex=[' Borough'], value='')
popdf['Region'] = popdf['Region'].replace(regex=[' Census Area'], value='')
popdf['Region'] = popdf['Region'].replace(regex=[' Parish'], value='')
popdf['Region'] = popdf['Region'].replace(regex=[' city'], value='')
popdf.to_csv('DATA/REFERENCE/us-population-by-county-' + revision + '.csv',index=False)

# STEP 4: Filter the data to only the US and calculate the daily metrics

# Create a dataframe to hold the data with the daily metrics
df3 = pd.DataFrame(columns=['Date','Region', 'Province', 'Country', 'Total_Confirmed', 'Total_Deaths'])

# Get a list of unique states (provinces)
provinces = pd.DataFrame(df2.Province.unique())
provinces = provinces.rename(columns={0:"Province"})

# Set the outbreak limits
death_limit = 0 #Assumption: Once the daily deaths exceeds this limit for the first time, community spread has started
confirmed_limit = 30 #Assumption: Once the confirmed cases exceeds this limit for the first time, community spread has started

# Loop through the states and calculate the daily metrics for each county
for pindex, p in provinces.iterrows():
    # Get a temporary data frame for this province
    sdf = df2[df2.Province == p.Province]
    sdf = sdf.sort_values(by=['Date'])
    sdf = sdf.reset_index(drop=True)
    
    # Filter the population down to this province
    rpopdf = popdf[popdf.Province == p.Province]
    rpopdf = rpopdf.reset_index(drop=True)
    
    # Merge the population data into the dataframe
    sdf = pd.merge(sdf, rpopdf[['Region', 'Population']], on='Region')

    # Get a list of unique regions (counties) for this province
    regions = pd.DataFrame(sdf.Region.unique())
    regions = regions.rename(columns={0:"Region"})

    for rindex, r in regions.iterrows():
        # Get a temporary dataframe for this region
        rdf = sdf[sdf.Region == r.Region]
        rdf = rdf.sort_values(by=['Date'])
        rdf = rdf.reset_index(drop=True)
        
        # Calculate the Daily Confirmed Cases
        rdf['Daily_Confirmed'] = rdf.Total_Confirmed - rdf.Total_Confirmed.shift(1)
        rdf['Daily_Deaths'] = rdf.Total_Deaths - rdf.Total_Deaths.shift(1)
        
        # Calculation the daily metrics per 100,000 people
        rdf['Proportional_Confirmed'] = rdf.Daily_Confirmed / rdf.Population * 100000
        rdf['Proportional_Deaths'] = rdf.Daily_Deaths / rdf.Population * 100000

        # Calculate the number of day from when the daily confirmed cases exceeds the outbreak limit
        if rdf.Daily_Confirmed.max() > confirmed_limit:
            # Get the first date when the number of confirmed cases exceeded 30
            cstart = rdf[rdf.Daily_Confirmed > confirmed_limit].iat[0,0]

            # Calculate the number of days from the start date (i.e. set the Days column values)
            rdf['Days_Confirmed'] = (rdf.Date - cstart).dt.days
        else: # If the maximum confirmed cases has not yet exceeded the limit then set the days to 0
            rdf['Days_Confirmed'] = 0

        # Calculate the number of day from when the daily confirmed cases exceeds the outbreak limit
        if rdf.Daily_Deaths.max() > death_limit:
            # Get the first date when the number of confirmed cases exceeded 30
            dstart = rdf[rdf.Daily_Deaths > death_limit].iat[0,0]

            # Calculate the number of days from the start date (i.e. set the Days column values)
            rdf['Days_Deaths'] = (rdf.Date - dstart).dt.days
        else: # If the maximum confirmed cases has not yet exceeded the limit then set the days to 0
            rdf['Days_Deaths'] = 0
            
        # Drop the first day of data because the calculations will be off
        rdf = rdf.sort_values(by=['Date'])
        rdf = rdf.reset_index(drop=True)
        rdf = rdf.drop(index=0)

        # Append to new data to the dataframe
        df3 = df3.append(rdf)

# Sort and reset the new dataframe
df3 = df3.sort_values(by=['Date'])
df3 = df3.reset_index(drop=True)
        
# Clean up the datatypes and precision for the new columns
df3.Daily_Confirmed = df3.Daily_Confirmed.fillna(0)
df3.Daily_Confirmed = df3.Daily_Confirmed.astype(int)
df3.Daily_Deaths = df3.Daily_Deaths.fillna(0)
df3.Daily_Deaths = df3.Daily_Deaths.astype(int)
df3.Days_Confirmed = df3.Days_Confirmed.astype(int)
df3.Days_Deaths = df3.Days_Deaths.astype(int)
df3.Population = df3.Population.astype(int)
df3 = df3.round({'Proportional_Confirmed':4,'Proportional_Deaths':4})
df3.to_csv('DATA/' + date.today().strftime('%Y-%m-%d') + '-COVID-19-US-' + revision + '.csv',index=False)
df3.to_csv('DATA/covid-19-data-usafacts+jhu-us-bycounty.csv',index=False)
df3